<a href="https://colab.research.google.com/github/Im-jDiOt/DeepLearning-Term-Proj/blob/feature-vgg/%7Bvggnet%7D_%7B10%7D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 # ============================================
# 1. 기본 설정
# ============================================
import pandas as pd
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')
import random # 추가

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from sklearn.model_selection import GroupKFold
import cv2
import os
from pathlib import Path

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("사용 디바이스:", device)

# Albumentations 설치 (Kaggle에는 기본 설치되어 있음)
import albumentations as A
from albumentations.pytorch import ToTensorV2

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 시드 고정 실행 (숫자는 아무거나 상관없으나 보통 42 사용)
seed_everything(42)
print("Seed set to 42 for reproducibility")


# ============================================
# 2. 경로 설정 (Kaggle 기준)
# ============================================
train_dir = "/kaggle/input/state-farm-distracted-driver-detection/imgs/train"
test_dir = "/kaggle/input/state-farm-distracted-driver-detection/imgs/test"
driver_csv_path = "/kaggle/input/state-farm-distracted-driver-detection/driver_imgs_list.csv"

# ============================================
# 3. 하이퍼파라미터
# ============================================
img_size = 224
# [수정] VGGNet은 메모리를 많이 사용하므로 32 -> 16으로 줄임 (OOM 방지)
batch_size = 16
num_classes = 10
num_epochs = 15
learning_rate = 3e-4
weight_decay = 1e-4
num_workers = 2


# ============================================
# 4. Dataset 정의
# ============================================
class DriverDataset(Dataset):
    def __init__(self, img_dir, df, driver_list, transform=None, is_test=False):
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
        self.images, self.labels = [], []

        if is_test:
            for img_name in sorted(os.listdir(img_dir)):
                self.images.append(os.path.join(img_dir, img_name))

        else:
            subset = df[df['subject'].isin(driver_list)]
            for _, row in subset.iterrows():
                class_name = row['classname']
                img_name = row['img']
                path = os.path.join(img_dir, class_name, img_name)
                self.images.append(path)
                self.labels.append(int(class_name[1:]))

        print(f"{'TEST' if is_test else 'TRAIN'}: {len(self.images)}개 이미지")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Albumentations는 resize를 transform 안에서 처리하는 게 좋지만,
        # 여기서는 원본 크기를 맞추기 위해 미리 resize
        img = cv2.resize(img, (img_size, img_size))

        if self.transform:
            img = self.transform(image=img)["image"]

        if self.is_test:
            return img, os.path.basename(img_path)
        else:
            return img, self.labels[idx]


# ============================================
# 5. Albumentations Transform (팀원 설정 유지)
# ============================================
train_transform = A.Compose([
    A.RandomScale(scale_limit=(-0.3, 0.0), p=1.0),  # 70~100% 크기
    A.PadIfNeeded(min_height=img_size, min_width=img_size, border_mode=cv2.BORDER_CONSTANT),
    A.RandomCrop(height=img_size, width=img_size, p=1.0),

    # [수정] 좌우반전 제거 (운전석 위치 혼동 방지)
    # A.HorizontalFlip(p=0.5),

    A.RandomBrightnessContrast(p=0.5),
    A.RandomGamma(gamma_limit=(80, 120), p=0.5),
    A.GaussianBlur(blur_limit=(3, 5), p=0.2),
    A.HueSaturationValue(p=0.3),

    A.Normalize(mean=(0.485,0.456,0.406),
                std=(0.229,0.224,0.225)),
    ToTensorV2()
])

eval_transform = A.Compose([
    A.Resize(img_size, img_size),
    A.Normalize(mean=(0.485,0.456,0.406),
                std=(0.229,0.224,0.225)),
    ToTensorV2()
])


# ============================================
# 6. GroupKFold split
# ============================================
df = pd.read_csv(driver_csv_path)
groups = df['subject']
gkf = GroupKFold(n_splits=5)
folds = list(gkf.split(df, df['classname'], groups))


# ============================================
# 7. Train / Validate 함수
# ============================================
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for x, y in tqdm(loader, desc="TRAIN", leave=False):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()

        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)
        correct += (out.argmax(1) == y).sum().item()
        total += y.size(0)

    return running_loss / total, 100 * correct / total


def validate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for x, y in tqdm(loader, desc="VALID", leave=False):
            x, y = x.to(device), y.to(device)

            out = model(x)
            loss = criterion(out, y)

            running_loss += loss.item() * x.size(0)
            correct += (out.argmax(1) == y).sum().item()
            total += y.size(0)

    return running_loss / total, 100 * correct / total


# ============================================
# 8. Warmup + CosineAnnealing Scheduler
# ============================================
class WarmupCosineLR(torch.optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, warmup_epochs, max_epochs, last_epoch=-1):
        self.warmup_epochs = warmup_epochs
        self.max_epochs = max_epochs
        self.base_lrs = [group['lr'] for group in optimizer.param_groups] # 초기화 추가
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        if self.last_epoch < self.warmup_epochs:
            return [base_lr * (self.last_epoch + 1) / self.warmup_epochs
                    for base_lr in self.base_lrs]

        cos_epoch = self.last_epoch - self.warmup_epochs
        total_cos = self.max_epochs - self.warmup_epochs

        return [
            base_lr * 0.5 * (1 + np.cos(np.pi * cos_epoch / total_cos))
            for base_lr in self.base_lrs
        ]


# ============================================
# 9. Fold 학습 (Fold 0만 실행)
# ============================================
fold_idx = 0
print(f"===== Start Training Fold {fold_idx} =====")

train_idx, val_idx = folds[fold_idx]

train_drivers = df.iloc[train_idx]['subject'].unique()
val_drivers   = df.iloc[val_idx]['subject'].unique()

train_dataset = DriverDataset(train_dir, df, train_drivers, transform=train_transform)
val_dataset   = DriverDataset(train_dir, df, val_drivers, transform=eval_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)


# ============================================
# 10. [수정됨] VGG16_BN 모델 정의
# ============================================
# 배치 정규화(BN)가 포함된 버전을 써야 학습이 잘 됩니다.
model = models.vgg16_bn(weights="IMAGENET1K_V1")

# VGG의 분류기는 'classifier'라는 이름의 Sequential 블록입니다.
# 마지막 레이어는 인덱스 6번입니다: Linear(in_features=4096, out_features=1000, bias=True)
num_ftrs = model.classifier[6].in_features # 4096
model.classifier[6] = nn.Linear(num_ftrs, num_classes) # 10개 클래스로 교체

model = model.to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = WarmupCosineLR(optimizer, warmup_epochs=3, max_epochs=num_epochs)

best_val_loss = 1e9
best_path = f"best_vgg16bn_fold{fold_idx}.pth"


# ============================================
# 11. Training Loop
# ============================================
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")

    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
    val_loss, val_acc = validate(model, val_loader, criterion)

    scheduler.step()

    print(f"Train Loss {train_loss:.4f} | Acc {train_acc:.2f}%")
    print(f"Val   Loss {val_loss:.4f} | Acc {val_acc:.2f}%")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_path)
        print(f"🔥 Best model updated: {best_path}")


# ============================================
# 12. 테스트셋 예측 & 제출 파일 생성
# ============================================
print("\nPredicting on Test Set...")
model.load_state_dict(torch.load(best_path, map_location=device))
model.eval()

test_dataset = DriverDataset(test_dir, None, None, transform=eval_transform, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

all_probs = []
all_imgs = []

with torch.no_grad():
    for imgs, names in tqdm(test_loader, desc="PREDICT"):
        imgs = imgs.to(device)
        out = model(imgs)
        probs = torch.softmax(out, dim=1).cpu().numpy()

        all_probs.append(probs)
        all_imgs.extend(names)

all_probs = np.vstack(all_probs)

submission = pd.DataFrame({
    'img': all_imgs,
    **{f'c{i}': all_probs[:, i] for i in range(10)}
})

sub_path = f"submission_vgg16bn_fold{fold_idx}.csv"
submission.to_csv(sub_path, index=False)

print("\nDONE:", sub_path)